<a href="https://colab.research.google.com/github/Yhawech/Exam--Practice/blob/master/Email_Spam_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import imaplib
import email
from email.header import decode_header
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# --- Gmail Login ---
IMAP_SERVER = "imap.gmail.com"
EMAIL_ACCOUNT = "prerakthaker62@gmail.com"
EMAIL_PASSWORD = "qjoo bpdn efsa gfzz"   # 16-digit App Password from Google

mail = imaplib.IMAP4_SSL(IMAP_SERVER)
mail.login(EMAIL_ACCOUNT, EMAIL_PASSWORD)
mail.select("inbox")

# --- Fetch latest 5 emails ---
status, messages = mail.search(None, "ALL")
email_ids = messages[0].split()[-5:]  # last 5 emails

emails_text = []
subjects = []

for e_id in email_ids:
    res, msg_data = mail.fetch(e_id, "(RFC822)")
    raw_msg = msg_data[0][1]
    msg = email.message_from_bytes(raw_msg)

    # Decode subject
    subject, encoding = decode_header(msg["Subject"])[0]
    if isinstance(subject, bytes):
        subject = subject.decode(encoding if encoding else "utf-8")
    subjects.append(subject)

    # Extract body
    body = ""
    if msg.is_multipart():
        for part in msg.walk():
            if part.get_content_type() == "text/plain":
                body += part.get_payload(decode=True).decode(errors="ignore")
    else:
        body = msg.get_payload(decode=True).decode(errors="ignore")

    emails_text.append(subject + " " + body)

# --- Simple Spam Classifier ---
X_train = ["Win a free prize", "Claim your lottery now",
           "Meeting agenda attached", "Let’s catch up tomorrow"]
y_train = [1, 1, 0, 0]  # 1 = spam, 0 = ham

vectorizer = TfidfVectorizer(stop_words="english")
X_train_vec = vectorizer.fit_transform(X_train)

model = MultinomialNB()
model.fit(X_train_vec, y_train)

# --- Predict each fetched email ---
for subject, text in zip(subjects, emails_text):
    X_test_vec = vectorizer.transform([text])
    prediction = model.predict(X_test_vec)[0]
    label = "🚨 SPAM" if prediction == 1 else "✅ Not Spam"
    print(f"Subject: {subject}\nPrediction: {label}\n{'-'*50}")


Subject: Welcome to TMDB!
Prediction: 🚨 SPAM
--------------------------------------------------
Subject: 😥 prerakthaker62, give us another chance? We’ve got thousands of new series and films.
Prediction: ✅ Not Spam
--------------------------------------------------
Subject: Security alert
Prediction: ✅ Not Spam
--------------------------------------------------
Subject: 2-Step Verification turned on
Prediction: ✅ Not Spam
--------------------------------------------------
Subject: Security alert
Prediction: ✅ Not Spam
--------------------------------------------------
